In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
def read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
        
    return content

In [3]:
def load_data():
    data = {}
    for index in range(3):
        X_train = pd.read_csv('Xtr%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_train = pd.read_csv('Xtr%s.csv' % index, names='1').values
        X_test = pd.read_csv('Xte%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_test = pd.read_csv('Xte%s.csv' % index, names='1').values
        labels = pd.read_csv('Ytr%s.csv' % index, names=('Id', 'Bound'), skiprows=[0], delimiter=',')

        data[index] = {
            'seq_train': seq_train,
            'X_train': X_train,
            'y_train': labels['Bound'].values,
            'seq_test': seq_test,
            'X_test': X_test,
            'ids': labels['Id'].values,
        }
        
    return data

In [4]:
from data_manipulation import split_train_test_valid, get_precision

## Kernel ridge regression (linear kernel)

In [5]:
from ridge_regression import get_ridge_prediction, kernel_ridge_regression

In [6]:
DATA = load_data()

train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['X_train'], DATA[0]['y_train'])
train_X.shape

(1200, 50)

In [7]:
def linear_K(x1, x2):
    return x1 @ x2.T

In [ ]:
REG_PARAMS_SPAN = [10**i for i in range(-10, 10)] + [10**i/2 for i in range(-10, 10)]

def get_best_reg_param():
    test_precisions = []
    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg)
        pred = get_ridge_prediction(linear_K, train_X, test_X, alpha)
        test_precisions.append(get_precision(pred, test_y))
        
    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
    return REG_PARAMS_SPAN[best_reg_index]

In [ ]:
reg_param = get_best_reg_param()
alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg_param)
pred = get_ridge_prediction(linear_K, train_X, valid_X, alpha)
get_precision(pred, valid_y)

## SVM (Linear Kernel)

In [8]:
from SVM import a_compute

In [106]:
train_y

array([1, 1, 0, ..., 1, 0, 0])

In [200]:
def SVM_train(kernel, X, y, Lambda):
    N = y.shape[0]
    y = 2*y-1
    
    K = kernel(X,X)
    alpha = np.zeros(N)
    # we assume K is of size N,N
    value = 100000

    for n in range(1,100):
        gradient = K @ alpha - y
        potential_alpha = alpha - 1/n * gradient
        for i in range(N):
            if potential_alpha[i] * y[i] > 1. / (2. * Lambda * N):
                potential_alpha[i] = (1. / (2. * Lambda * N))
            if potential_alpha[i] * y[i] < 0.:
                potential_alpha[i] = 0.
        alpha = potential_alpha

    return alpha

def SVM_predict(X_train, X_test, alpha):
    pos_neg = np.sign(alpha @ linear_K(train_X, test_X))
    print(pos_neg)
    return (pos_neg/2+0.5).astype(int)

In [201]:
K = linear_K(train_X, train_X)
alpha = SVM_train(linear_K, train_X, train_y, 0.1)
alpha

array([ 0.00416667,  0.00416667,  0.        , ...,  0.00416667,
        0.        ,  0.        ])

In [202]:
alpha @ linear_K(train_X, test_X)

array([ 0.05709082,  0.05511874,  0.05492872,  0.06023797,  0.05724638,
        0.05634206,  0.05595266,  0.05648728,  0.05314026,  0.05368915,
        0.05871584,  0.05477562,  0.05104019,  0.0562308 ,  0.05624458,
        0.05596448,  0.053731  ,  0.05580941,  0.05592559,  0.05472885,
        0.05585765,  0.05376349,  0.05113372,  0.05624557,  0.05561102,
        0.05886303,  0.05459003,  0.05415879,  0.05567305,  0.05258694,
        0.05151819,  0.05653897,  0.05713463,  0.05526347,  0.05967677,
        0.05455606,  0.05590196,  0.05409922,  0.05543774,  0.05558197,
        0.05443693,  0.0525126 ,  0.05682794,  0.05575526,  0.0571622 ,
        0.05570603,  0.05536931,  0.05562628,  0.0580611 ,  0.05716515,
        0.05539097,  0.05432764,  0.05311417,  0.05846723,  0.05740932,
        0.05637258,  0.0550508 ,  0.05547515,  0.0546043 ,  0.05638882,
        0.0541327 ,  0.05545349,  0.05324216,  0.05275333,  0.05434438,
        0.05417208,  0.05727493,  0.05650894,  0.05540131,  0.05

In [195]:
pred = SVM_predict(train_X, test_X, alpha)
get_precision(pred, test_y)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

0.495

In [196]:
print(np.min(alpha))
print(np.max(alpha))


0.0
0.00416666666667


In [197]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

## Spectrum kernel 

In [ ]:
train_seq, train_y, test_seq, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['seq_train'], DATA[0]['y_train'])

In [ ]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['X_train'], DATA[0]['y_train'])
train_X.shape

In [ ]:
from spectrum_kernel import transform_to_index_and_save, get_words

In [ ]:
def load_data_k(k):
    data = {}
    for index in range(3):
        X_train = np.loadtxt('spectral_preindexed/Xtr%s_spectral_%s.gz' % (index, k))
        seq_train = pd.read_csv('Xtr%s.csv' % index, names='1').values
        X_test = np.loadtxt('spectral_preindexed/Xte%s_spectral_%s.gz' % (index, k))
        seq_test = pd.read_csv('Xte%s.csv' % index, names='1').values
        labels = pd.read_csv('Ytr%s.csv' % index, names=('Id', 'Bound'), skiprows=[0], delimiter=',')

        X_train_mean = X_train.mean(axis=0)
        X_train_center = X_train - X_train_mean
        X_test_center = X_test - X_train_mean
        
        data[index] = {
            'seq_train': seq_train,
            'X_train': X_train,
            'y_train': labels['Bound'].values,
            'seq_test': seq_test,
            'X_test': X_test,
            'ids': labels['Id'].values,
        }
        
    return data

In [ ]:
k = 10
def spectrum_2(word1, word2):
    return sum(word2.count(word1[i:i+k]) for i in range(len(word1)-k+1))

def spectrum_kernel_2(seq1, seq2):
    n1 = seq1.shape[0]
    n2 = seq2.shape[0]

    K = np.zeros((n1,n2))
    for i in range(n1):
        for j in range(n2):
            K[i,j] = spectrum_2(seq1[i,0], seq2[j,0])
    
    return K

K = spectrum_kernel_2(train_seq, train_seq)

In [ ]:
np.sum(K[0,:])

In [ ]:
def spectrum_kernel(x1, x2):        
    return x1 @ x2.T

In [ ]:
K = spectrum_kernel(train_X, train_X)
K

In [ ]:
REG_PARAMS_SPAN = [10**i for i in range(-5, 5)] + [10**i/2 for i in range(-5, 5)]

def get_best_reg_param(train_X, train_y, test_X, test_y):
    test_precisions = []
    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regression(spectrum_kernel, train_X, train_y, reg)
        pred = get_ridge_prediction(spectrum_kernel, train_X, test_X, alpha)
        test_precisions.append(get_precision(pred, test_y))
        
    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
    return REG_PARAMS_SPAN[best_reg_index], test_precisions[best_reg_index]

In [ ]:
get_best_reg_param(train_X, train_y, test_X, test_y)

## Mismatch Kernel 

In [ ]:
import Levenshtein

In [ ]:
DATA = load_data_k(6)

In [ ]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['X_train'], DATA[0]['y_train'])
train_X.shape

In [ ]:
def get_mismatch_matrix(k,m):
    words = get_words(k)
    mismatch_matrix = np.zeros((len(words), len(words)))
    for i in range(len(words)):
        for j in range(i, len(words)):
            if Levenshtein.hamming(words[i], words[j]) <= m:
                mismatch_matrix[i,j] = 1/2
                mismatch_matrix[j,i] = 1/2
    return mismatch_matrix

mismatch_matrix = get_mismatch_matrix(6,1)

In [ ]:
def mismatch_kernel(x1, x2):
    return x1 @ mismatch_matrix @ x2.T

In [ ]:
K = mismatch_kernel(train_X, train_X)
K

In [ ]:
REG_PARAMS_SPAN = [10**i for i in range(-10, 10)] + [10**i/2 for i in range(-10, 10)]

def get_best_reg_param(train_X, train_y, test_X, test_y):
    test_precisions = []
    K_train = mismatch_kernel(train_X, train_X)
    K_test = mismatch_kernel(train_X, test_X)
    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regression(mismatch_kernel, train_X, train_y, reg, K=K_train)
        pred = get_ridge_prediction(mismatch_kernel, train_X, test_X, alpha, K_x=K_test)
        test_precisions.append(get_precision(pred, test_y))
        
    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
    return REG_PARAMS_SPAN[best_reg_index], test_precisions[best_reg_index]

In [ ]:
get_best_reg_param(train_X, train_y, test_X, test_y)

## Homemade mismatch 

In [ ]:
def get_exp_mismatch_matrix(k, _lambda):
    words = get_words(k)
    exp_mismatch_matrix = np.zeros((len(words), len(words)))
    for i in range(len(words)):
        exp_mismatch_matrix[i,i] = 1
        for j in range(i+1, len(words)):
            exp_mismatch_matrix[i,j] = _lambda**Levenshtein.hamming(words[i], words[j])
            exp_mismatch_matrix[j,i] = exp_mismatch_matrix[i,j]
            
    return exp_mismatch_matrix

exp_mismatch_matrix = get_exp_mismatch_matrix(6,0.5)

In [ ]:
exp_mismatch_matrix[0,:]

In [ ]:
def exp_mismatch_kernel(x1, x2):
    return x1 @ exp_mismatch_matrix @ x2.T

In [ ]:
K = exp_mismatch_kernel(train_X, train_X)
K

In [ ]:
REG_PARAMS_SPAN = [10**i for i in range(0, 10)]
LAMBDA_SPAN = [0.5, 0.1, 0.05, 0.01]

def get_best_reg_param(train_X, train_y, test_X, test_y):
    test_precisions = []
    K_train = mismatch_kernel(train_X, train_X)
    K_test = mismatch_kernel(train_X, test_X)

    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regression(mismatch_kernel, train_X, train_y, reg, K=K_train)
        pred = get_ridge_prediction(mismatch_kernel, train_X, test_X, alpha, K_x=K_test)
        test_precisions.append(get_precision(pred, test_y))

    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
        
    return REG_PARAMS_SPAN[best_reg_index], test_precisions[best_reg_index]

In [ ]:
get_best_reg_param(train_X, train_y, test_X, test_y)

## Substring Kernel 

In [ ]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['seq_train'], DATA[0]['y_train'])
train_X.shape

In [ ]:
from substring_kernel import K_k, substring_kernel

In [ ]:
LAMBDA = 0.1
K_k(LAMBDA, 2, 'car', 'car') - 2*LAMBDA**4+LAMBDA**6 < 10**-5

In [ ]:
K_k(0.1, 5, 
   'CAGCTTTTATCACCTTTGAGGGAAAGTCATATTAATTTAATACTGCACACACTTGTACAACAGATCTTCTTTACTATTAAAACTCAGTTTATCAAATCACA',
   'AATAACATACCCCACTCTTTCATCTCAATCAAAAATTGAAAAAGTCAAAGAATCCTGCTTTTTTGTTTTTCTCCAAGCCATTACCCCCTCTTGATCATTGC'
   )

In [ ]:
# This is too slow by several orders of magnitude...
for j in range(train_X.shape[0]):
    print(j)
    K_k(0.1, 2, train_X[1,0], train_X[j,0])

## Final results

In [ ]:
DATA = load_data_k(4)

In [ ]:
DATA[0]['X_train'].shape

In [ ]:
def produce_results(preds, ids):
    data = [
        '%s,%s' % (id, pred) for pred, id in zip(preds, ids)
    ]
    
    data.insert(0, 'Id,Bound')
    with open('submission.csv', 'w') as f:
        f.write('\n'.join(data))

In [ ]:
def train_tune_and_pred_on_test():
    preds = []
    ids = []
    for index in range(3):
        train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(
            DATA[index]['X_train'], 
            DATA[index]['y_train']
        )

        # Finding best parameter
        reg_param, _ = get_best_reg_param(train_X, train_y, test_X, test_y)

        # Validation precision
        alpha = kernel_ridge_regression(spectrum_kernel, train_X, train_y, reg_param)
        pred = get_ridge_prediction(spectrum_kernel, train_X, valid_X, alpha)
        print("Dataset %s has found a parameter (%s) with validation precision %.3f" % (index, reg_param, get_precision(pred, valid_y)))
        
        continue
        # Kaggle submission
        alpha = kernel_ridge_regression(exp_mismatch_kernel, DATA[index]['X_train'], DATA[index]['y_train'], reg_param)
        pred = get_ridge_prediction(exp_mismatch_kernel, DATA[index]['X_train'], DATA[index]['X_test'], alpha)
        
        preds += list(pred)

    produce_results(preds, range(3000))

In [ ]:
train_tune_and_pred_on_test()

In [ ]:
def see_variability(reg_params, N=20):
    results = np.zeros((3,N))
    for index in range(3):
        print(index)
        for n in range(N):
            train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(
                DATA[index]['X_train'], 
                DATA[index]['y_train']
            )
            
            # Validation precision
            alpha = kernel_ridge_regression(spectrum_kernel, train_X, train_y, reg_params[index])
            pred = get_ridge_prediction(spectrum_kernel, train_X, valid_X, alpha)
            results[index,n] = get_precision(pred, valid_y)
    return results

res = see_variability(reg_params=[1000,500,1000])

In [ ]:
plt.boxplot(res.T)
plt.show()

In [ ]:
res.mean(axis=1)